In [27]:
import pandas as pd
import datatable as dt

In [2]:
# dataset = dt.fread("/home/dilyara.baymurzina/markov_dataset/train_markov.csv")
# dataset.to_jay("./datasets/russian_chats/train_markov.jay")

In [16]:
original_path = "/home/dilyara.baymurzina/markov_dataset/preprocessed_ru_convers_tokenized.txt"
part_dataset = "./datasets/russian_chats/part_markov.txt"
dataset_size = 10000
with open(part_dataset, 'w') as part_dataset:
    with open(original_path, 'r') as original_dataset:
        part_dataset_str = "".join([next(original_dataset) for _ in range(dataset_size)])
        part_dataset.write(part_dataset_str)

In [12]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained('tinkoff-ai/response-quality-classifier-large')
model = AutoModelForSequenceClassification.from_pretrained('tinkoff-ai/response-quality-classifier-large')
inputs = tokenizer('[CLS]привет[SEP]привет![SEP]как дела?[RESPONSE_TOKEN]великолепно, просто охуительно, лучше не куда, а у тебя?', max_length=514, add_special_tokens=False, return_tensors='pt')
with torch.inference_mode():
    logits = model(**inputs).logits
    probas = torch.sigmoid(logits)[0].cpu().detach().numpy()
relevance, specificity = probas
relevance, specificity


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


(0.8152041, 0.667545)

In [14]:
class RelevanceRanker:
    def __init__(self, model_name='tinkoff-ai/response-quality-classifier-large'):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForSequenceClassification.from_pretrained(model_name)
    
    def rank(self, context, response):
        history = "[SEP]".join(context)
        input_string = f'[CLS]{history}[RESPONSE_TOKEN]{response}'
        inputs = self.tokenizer(
            input_string, 
            max_length=512, 
            add_special_tokens=False, 
            return_tensors='pt'
        )
        with torch.inference_mode():
            logits = self.model(**inputs).logits
            probas = torch.sigmoid(logits)[0].cpu().detach().numpy()
        relevance, specificity = probas
        return relevance, input_string

relevance_ranker = RelevanceRanker()

### split dialogues into groups

In [2]:
original_dialog_1 = """
Ну , вообще – то , голосовые команды наверняка дублировались жестами .
Так что небольшой обман в видео все – таки есть .
простите , вы кинолог ?
Хотя бы личную собаку дрессировали ?
Знаете зачем жесты показывают вместе с командой ?
Вовсе не потому , что собака не понимает слов .
У моего друга была овчарка , которую он нашел на улице .
Сам он ничего не знал о дрессуре .
Просто разговаривал с ней , как с человеком и она делала то , что он просит .
Пруф ?
Никакого .
Давно это было .
Даже видеокамеры в те времена мало у кого были .
А сама Айна уже умерла .
И сын ее Чак , помер .
Однажды я захотел проверить , в самом деле Айна понимает слова человеческие или ориентируется на интонацию или что – то еще .
Я пришел в гараж , который она охраняла ночью и увидел , что она меня не дождалась и сделала кучу .
Я стал гладить ее и ласково говорить : — " Айна , а кто это у нас тут насрал ?"
Услышав последнее слово , она в страхе припала к полу и стала заползать под машину , ожидая наказания .
Так я убедился , что она воспринимает именно само слово , а не жесты или интонацию голоса .
Потом я сам заводил собак и узнал , что это действительно так .
Собаки запоминают слова и даже отличают очень похожие .
Последняя моя собака могла отличать , когда я обращаюсь к ней , а когда рассказываю кому – то про нее в контексте .
То есть не реагировала на свое имя , если я просто говорю про нее , а не обращаюсь к ней .
"""

original_dialog_1_manual = """
Ну , вообще – то , голосовые команды наверняка дублировались жестами . Так что небольшой обман в видео все – таки есть .

Простите , вы кинолог ? Хотя бы личную собаку дрессировали ? Знаете зачем жесты показывают вместе с командой ? Вовсе не потому , что собака не понимает слов . У моего друга была овчарка , которую он нашел на улице . Сам он ничего не знал о дрессуре . Просто разговаривал с ней , как с человеком и она делала то , что он просит .

Пруф ? 

Никакого . Давно это было . Даже видеокамеры в те времена мало у кого были . А сама Айна уже умерла . И сын ее Чак , помер . Однажды я захотел проверить , в самом деле Айна понимает слова человеческие или ориентируется на интонацию или что – то еще . Я пришел в гараж , который она охраняла ночью и увидел , что она меня не дождалась и сделала кучу . Я стал гладить ее и ласково говорить : — " Айна , а кто это у нас тут насрал ?" Услышав последнее слово , она в страхе припала к полу и стала заползать под машину , ожидая наказания . Так я убедился , что она воспринимает именно само слово , а не жесты или интонацию голоса . Потом я сам заводил собак и узнал , что это действительно так . Собаки запоминают слова и даже отличают очень похожие . Последняя моя собака могла отличать , когда я обращаюсь к ней , а когда рассказываю кому – то про нее в контексте . То есть не реагировала на свое имя , если я просто говорю про нее , а не обращаюсь к ней .
"""

original_dialog_2 = """
А я считаю , очень правильно .
Я вообще не понимаю , почему на работу , например , могут отказать женщине 26 – 35 лет , если у неё нет детей .
Должны рассматривать всех и выбирать уже , исходя из профессиональных качеств , а не смотреть сколько детей , какой национальности и пола .
Хотя бы из – за того , что она не сможет упаковать 46 " ящик и выдать покупателю .
Почему не сможет ?
Да пофиг .
По КЗОТу вроде не более 10 кг .
Иначе нарушение закона работодателем .
"""

original_dialog_2_manual = """
А я считаю , очень правильно . 
Я вообще не понимаю , почему на работу , например , могут отказать женщине 26 – 35 лет , если у неё нет детей . Должны рассматривать всех и выбирать уже , исходя из профессиональных качеств , а не смотреть сколько детей , какой национальности и пола .
Хотя бы из – за того , что она не сможет упаковать 46 " ящик и выдать покупателю .
Да пофиг .
По КЗОТу вроде не более 10 кг . Иначе нарушение закона работодателем .
"""

In [20]:
dialog = [item for item in original_dialog_2.split('\n') if item]
dialog_len = 1
temp_dialog_len = 0
for i in range(dialog_len, len(dialog) - dialog_len+1):
    part = dialog[i - dialog_len:i + dialog_len]
    context = part[:dialog_len]
    response = part[-1]
    # print("@@@".join(part))
    # print(context)
    # print(response)
    relevance, input_string = relevance_ranker.rank(context, response)
    print(F"INPUT_STRING: {input_string}")
    print(f"RELEVANCE: {relevance}")
    print()

INPUT_STRING: [CLS]А я считаю , очень правильно .[RESPONSE_TOKEN]Я вообще не понимаю , почему на работу , например , могут отказать женщине 26 – 35 лет , если у неё нет детей .
RELEVANCE: 0.9085078239440918

INPUT_STRING: [CLS]Я вообще не понимаю , почему на работу , например , могут отказать женщине 26 – 35 лет , если у неё нет детей .[RESPONSE_TOKEN]Должны рассматривать всех и выбирать уже , исходя из профессиональных качеств , а не смотреть сколько детей , какой национальности и пола .
RELEVANCE: 0.9774068593978882

INPUT_STRING: [CLS]Должны рассматривать всех и выбирать уже , исходя из профессиональных качеств , а не смотреть сколько детей , какой национальности и пола .[RESPONSE_TOKEN]Хотя бы из – за того , что она не сможет упаковать 46 " ящик и выдать покупателю .
RELEVANCE: 0.8563143014907837

INPUT_STRING: [CLS]Хотя бы из – за того , что она не сможет упаковать 46 " ящик и выдать покупателю .[RESPONSE_TOKEN]Почему не сможет ?
RELEVANCE: 0.9779601693153381

INPUT_STRING: [CLS]П

In [21]:
dialog = [item for item in original_dialog_1.split('\n') if item]
dialog_len = 1
temp_dialog_len = 0
for i in range(dialog_len, len(dialog) - dialog_len+1):
    part = dialog[i - dialog_len:i + dialog_len]
    context = part[:dialog_len]
    response = part[-1]

    relevance, input_string = relevance_ranker.rank(context, response)
    print(F"INPUT_STRING: {input_string}")
    print(f"RELEVANCE: {relevance}")
    print()

INPUT_STRING: [CLS]Ну , вообще – то , голосовые команды наверняка дублировались жестами .[RESPONSE_TOKEN]Так что небольшой обман в видео все – таки есть .
RELEVANCE: 0.9836452007293701

INPUT_STRING: [CLS]Так что небольшой обман в видео все – таки есть .[RESPONSE_TOKEN]простите , вы кинолог ?
RELEVANCE: 0.8198174834251404

INPUT_STRING: [CLS]простите , вы кинолог ?[RESPONSE_TOKEN]Хотя бы личную собаку дрессировали ?
RELEVANCE: 0.7496459484100342

INPUT_STRING: [CLS]Хотя бы личную собаку дрессировали ?[RESPONSE_TOKEN]Знаете зачем жесты показывают вместе с командой ?
RELEVANCE: 0.3005693554878235

INPUT_STRING: [CLS]Знаете зачем жесты показывают вместе с командой ?[RESPONSE_TOKEN]Вовсе не потому , что собака не понимает слов .
RELEVANCE: 0.9718955159187317

INPUT_STRING: [CLS]Вовсе не потому , что собака не понимает слов .[RESPONSE_TOKEN]У моего друга была овчарка , которую он нашел на улице .
RELEVANCE: 0.5639684796333313

INPUT_STRING: [CLS]У моего друга была овчарка , которую он нашел

### Итог. Эта модель не работает совсем

### Разбиение датасета

#### перевод в jay

### original dataset

In [36]:
original_path = "/home/dilyara.baymurzina/markov_dataset/preprocessed_ru_convers_tokenized.txt"
dialogues = []
with open(original_path, 'r') as original_dataset:
    prev_content = ""
    dialog = []
    for content in original_dataset:
        if len(content) == 1:
            dialog = "@@@".join(dialog)
            dialogues.append(dialog)
            dialog = []
        else:
            prev_content = content
            dialog.append(content.strip())
dt.Frame({
	"dialogue": dialogues,
}
).to_jay("./datasets/russian_chats/markov_dialogues.jay")

In [37]:
dataset = dt.fread("./datasets/russian_chats/markov_dialogues.jay")

In [69]:
dataset

,dialogue
,▪▪▪▪▪▪▪▪
0,А вообще уже было Здесь ещё точно не было !@@@Я по…
1,"Я думал в / kucha . d3 . ru / лулзы ловят , а полу…"
2,это чё за хоббит ?@@@а ... понял ... беззубый хобб…
3,Это какой – то ... позор !( с ) Бывает .@@@Социали…
4,Но зачем ?@@@Девчонки ведутся !@@@Логично .@@@Мне …
5,"это действительно не сложно , я за вечер научился …"
6,"И как её гладить после стирки ?@@@вот – вот , деву…"
7,"У него есть все шансы найти себе подругу .@@@Да , …"
8,Видимо индийцы с цыганами всё – таки родственники …


In [57]:
dataset[1002, 0].split("@@@")

['Проповедники приперлись в парк , где в жаркий день отдыхали люди , и получили эмоциональный ответ .',
 'Автор бредит гомосЭксуалистами .',
 'Почитайте описание видео , пожалуйста .',
 'У меня свои глаза и уши .',
 'Не надо искать заговор и фобию там , где её нет .',
 'Ну вы же сами так подали свой пост .',
 'И потом , какая разница какой секс – ориентации хулиган ?']

In [61]:
dialog_1 = dataset[1002, 0].split("@@@")
print("\n".join(dialog_1))
aug_dialog_1 = []

for i in range(1, len(dialog_1), 2):
    context = dialog_1[:i]
    context = " [SEP] ".join(context)
    response = dialog_1[i]
    print(f"CONTEXT: {context} RESPONSE: {response}")

Проповедники приперлись в парк , где в жаркий день отдыхали люди , и получили эмоциональный ответ .
Автор бредит гомосЭксуалистами .
Почитайте описание видео , пожалуйста .
У меня свои глаза и уши .
Не надо искать заговор и фобию там , где её нет .
Ну вы же сами так подали свой пост .
И потом , какая разница какой секс – ориентации хулиган ?
CONTEXT: Проповедники приперлись в парк , где в жаркий день отдыхали люди , и получили эмоциональный ответ . RESPONSE: Автор бредит гомосЭксуалистами .
CONTEXT: Проповедники приперлись в парк , где в жаркий день отдыхали люди , и получили эмоциональный ответ . [SEP] Автор бредит гомосЭксуалистами . [SEP] Почитайте описание видео , пожалуйста . RESPONSE: У меня свои глаза и уши .
CONTEXT: Проповедники приперлись в парк , где в жаркий день отдыхали люди , и получили эмоциональный ответ . [SEP] Автор бредит гомосЭксуалистами . [SEP] Почитайте описание видео , пожалуйста . [SEP] У меня свои глаза и уши . [SEP] Не надо искать заговор и фобию там , где е

In [86]:
!ls -l ./datasets/russian_chats/ -h

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
total 11G
-rw-rw-r-- 1 kosenko kosenko  6.9G Dec 15 21:03 augumented_markov_dialogues.jay
-rw-rw-r-- 1 kosenko kosenko  3.1G Dec 15 19:54 markov_dialogues.jay
-rw-rw-r-- 1 kosenko kosenko 1016K Dec 15 19:29 part_markov.txt
-rwxrwxrwx 1 kosenko kosenko  131M Dec 15 00:33 telegram_chats.zip
-rw-rw-r-- 1 kosenko kosenko   168 Dec 15 19:44 test.jay


### convert original dataset for training

In [85]:
dialogues = []

dataset_len = dataset[:, dt.count()][0, 0]

for pos in range(300_000):
    dialog_sample = dataset[pos, 0].split("@@@")

    for i in range(1, len(dialog_sample), 2):
        context = dialog_sample[:i]
        context = " [SEP] ".join(context)
        response = dialog_sample[i]
        
        dialogues.append({
            "context": context,
            "response": response
        })
        


dt.Frame(dialogues).to_jay("./datasets/russian_chats/augumented_markov_dialogues.jay")